# Natural Language Processing

## Importing the libraries

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import nltk
import os
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
import zipfile

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv
/kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip


## Importing the dataset

In [3]:
zf1 = zipfile.ZipFile("/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip")
train = pd.read_csv(zf1.open("labeledTrainData.tsv"), sep="\t")
zf2 = zipfile.ZipFile("/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip")
test = pd.read_csv(zf2.open("testData.tsv"), sep="\t")

In [4]:
train.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [5]:
test.head()

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [6]:
train.shape, test.shape

((25000, 3), (25000, 2))

In [7]:
dataset=pd.concat([train, test], axis=0, sort=False)
dataset.shape

(50000, 3)

In [8]:
len(dataset)

50000

In [9]:
dataset["review"].iloc[1]

'\\The Classic War of the Worlds\\" by Timothy Hines is a very entertaining film that obviously goes to great effort and lengths to faithfully recreate H. G. Wells\' classic book. Mr. Hines succeeds in doing so. I, and those who watched his film with me, appreciated the fact that it was not the standard, predictable Hollywood fare that comes out every year, e.g. the Spielberg version with Tom Cruise that had only the slightest resemblance to the book. Obviously, everyone looks for different things in a movie. Those who envision themselves as amateur \\"critics\\" look only to criticize everything they can. Others rate a movie on more important bases,like being entertained, which is why most people never agree with the \\"critics\\". We enjoyed the effort Mr. Hines put into being faithful to H.G. Wells\' classic novel, and we found it to be very entertaining. This made it easy to overlook what the \\"critics\\" perceive to be its shortcomings."'

### Cleaning the texts

In [10]:
#nltk.download('stopwords')

stops = stopwords.words('english')
#indNot = stops.index("not")
#del(stops[indNot])

In [11]:

corpus = []
for i in range(0, len(dataset)):
    review = re.sub('[^a-zA-Z]', ' ', dataset['review'].iloc[i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stops)]
    review = ' '.join(review)
    corpus.append(review)

### understanding of loop

In [12]:
'''i = 8
review = re.sub('[^a-zA-Z]', ' ', dataset['Review'].iloc[i])
review = review.lower()
review = review.split()
ps = PorterStemmer()
review = [ps.stem(word) for word in review if not word in set(stops)]
review = ' '.join(review)
corpus.append(review)'''

"i = 8\nreview = re.sub('[^a-zA-Z]', ' ', dataset['Review'].iloc[i])\nreview = review.lower()\nreview = review.split()\nps = PorterStemmer()\nreview = [ps.stem(word) for word in review if not word in set(stops)]\nreview = ' '.join(review)\ncorpus.append(review)"

In [13]:
#corpus

In [14]:
cv1 = CountVectorizer(max_features = 1500)
X1 = cv1.fit_transform(corpus).toarray()
#y = dataset.iloc[:, 1]

In [15]:
cv2 = TfidfVectorizer(max_features = 1500)
X2 = cv2.fit_transform(corpus).toarray()
#y = dataset.iloc[:, 1]

In [16]:
#X1

In [17]:
X_train1=X1[:25000]
y_train1=dataset["sentiment"].iloc[:25000]
X_test1=X1[25000:]

In [18]:
X_train2=X2[:25000]
y_train2=dataset["sentiment"].iloc[:25000]
X_test2=X2[25000:]

### Fitting Naive Bayes to the Training set

In [19]:
# Fitting Naive Bayes to the Training set

classifier1 = GaussianNB()
classifier1.fit(X_train1, y_train1)


GaussianNB(priors=None, var_smoothing=1e-09)

In [20]:
classifier2 = GaussianNB()
classifier2.fit(X_train2, y_train2)

GaussianNB(priors=None, var_smoothing=1e-09)

### prediction

In [21]:
# Predicting the Test set results
y_pred1 = classifier1.predict(X_test1)

In [22]:
y_pred2 = classifier2.predict(X_test2)

### to_csv

In [23]:
mysubmission2=pd.read_csv("/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv")
mysubmission2["sentiment"]=y_pred2
mysubmission2.to_csv("mysubmission2.csv", index=False)

In [24]:
mysubmission=pd.read_csv("/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv")
mysubmission["sentiment"]=y_pred1
mysubmission.to_csv("mysubmission.csv", index=False)